<a href="https://colab.research.google.com/github/GeorgeTelles/Backtest_Bollinger_Bands/blob/main/Month_Patterns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
  <img src="https://raw.githubusercontent.com/GeorgeTelles/georgetelles/f69531ec6b293b5148563588a764c010015d315e/logo_clara.png" alt="logo clara" width="300" style="display: inline-block; vertical-align: top; margin-right: 10px;">
  <img src="https://raw.githubusercontent.com/GeorgeTelles/georgetelles/f69531ec6b293b5148563588a764c010015d315e/logo_dark.png" alt="logo dark" width="300" style="display: inline-block; vertical-align: top;">
</div>

# Monthly Pattern Analyzer



## Description

The **Monthly Pattern Analyzer** is a project designed to identify and analyze performance patterns of stocks based on the months of the year. This algorithm examines historical data to determine if there are recurring trends or behaviors in specific months, such as January, February, March, and so on. The goal is to identify months that exhibit higher gains or losses in stock performance.

## Features

- **Data Import**: Retrieves historical market data using the MetaTrader5 library.
- **Pattern Analysis**: Assesses stock performance for each month of the year.
- **Outlier Handling**: Filters outliers to enhance the accuracy of monthly pattern analysis.
- **Visualization and Export**: Consolidates results into a DataFrame and exports them to an Excel file for detailed analysis and review.

## Workflow

1. **Import Libraries**: Load the necessary libraries for data processing and analysis.
2. **Create Asset List**: Define the assets to be analyzed, which may include specific stocks or indices.
3. **Set Start Date**: Establish the start date for analyzing historical data.
4. **Calculate Results**: Implements a function to:
   - Initialize the MetaTrader5 connection.
   - Retrieve historical data for the specified time periods.
   - Calculate performance metrics for each month of the year.
   - Remove outliers for more accurate pattern analysis.
   - Aggregate data by month and calculate relevant statistics.
5. **Process Data**: Execute the calculation function for all specified assets and time periods, consolidating the results into a DataFrame.
6. **Export Results**: Export the analyzed data to an Excel file for easy review and detailed analysis.

## Usage

1. **Install Dependencies**: Ensure all necessary libraries are installed.
2. **Define Asset List**: Configure the list of assets for analysis, including specific stocks or indices.
3. **Run the Script**: Execute the script to analyze stock performance by month.
4. **Review and Export Results**: Results will be consolidated and exported to an Excel file for detailed review and analysis.

## Contributing

Contributions are welcome! If you have suggestions for improvements or want to add new features, feel free to open an issue or submit a pull request.

## License

This project is licensed under the [MIT License](LICENSE).

#Instalação e Improtação de Biliotecas

In [ ]:
!pip install tradingcomdados

In [ ]:
import yfinance as yf
from datetime import datetime
import pandas as pd
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import os
from tradingcomdados import alternative_data as ad

**Lista IBOV**

In [ ]:
lista_ativos = [codigo + '.SA' for codigo in ad.index_composition('ibov', mode = 'list')]

**Lista Personalizada**

In [ ]:
lista_ativos = ["PETR4.SA",]

In [ ]:
# Definir tradução de meses
traducao_meses = {
    1: 'Janeiro',
    2: 'Fevereiro',
    3: 'Março',
    4: 'Abril',
    5: 'Maio',
    6: 'Junho',
    7: 'Julho',
    8: 'Agosto',
    9: 'Setembro',
    10: 'Outubro',
    11: 'Novembro',
    12: 'Dezembro'
}



In [ ]:
# Inicializar listas globais
empresas_globais = []
meses_globais = []
resultados_globais = []
holds = []

for ativo in lista_ativos:
    try:
        # Extrair dados do Yahoo Finance
        stock_data = yf.download(ativo, start="2013-01-01", interval="1mo")
        stock_data['Mes'] = stock_data.index.month
        stock_data['Variacao_Porcentagem'] = ((stock_data['Close'] - stock_data['Open']) / stock_data['Open']) * 100
        # Calcular a variação percentual desde o início do período até o final
        hold = ((stock_data['Adj Close'].iloc[-1] - stock_data['Adj Close'].iloc[0]) / stock_data['Adj Close'].iloc[0]) * 100

         #Tratando os outliers:
        mean = stock_data['Variacao_Porcentagem'].mean()
        std_dev = stock_data['Variacao_Porcentagem'].std()
        lower_bound = mean - 3 * std_dev
        upper_bound = mean + 3 * std_dev

        # Atualizar rates_frame para remover outliers
        stock_data = stock_data[
            (stock_data['Variacao_Porcentagem'] >= lower_bound) &
            (stock_data['Variacao_Porcentagem'] <= upper_bound)
        ]

        grupo_mes = stock_data.groupby('Mes')['Variacao_Porcentagem'].sum()
        group_data_filtrado = pd.DataFrame()

        # Inicializar listas temporárias para cada ativo
        empresas_temp = []
        meses_temp = []
        resultados_temp = []
        holds_temp = []

        grupo_mes_filtrado = stock_data.groupby('Mes')['Variacao_Porcentagem'].sum()

        # Loop sobre o grupo filtrado
        for mes, resultado in zip(grupo_mes_filtrado.index, grupo_mes_filtrado):
            mes_traduzido = traducao_meses[mes]
            resultado_formatado = "{:.2f}".format(resultado)  # Formatação com duas casas decimais

            empresas_temp.append(ativo)
            meses_temp.append(mes_traduzido)
            resultados_temp.append(resultado_formatado)
            holds_temp.append(hold)

        # Estender as listas globais com as listas temporárias
        empresas_globais.extend(empresas_temp)
        meses_globais.extend(meses_temp)
        resultados_globais.extend(resultados_temp)
        holds.extend(holds_temp)

    except Exception as e:
        print(f"Erro ao processar {ativo}: {e}")

# Criação do DataFrame com as listas globais
df_resultados = pd.DataFrame({
    'Empresa': empresas_globais,
    'Mes': meses_globais,
    'Resultado': resultados_globais,
    "Hold": holds
})

df_resultados['Resultado'] = pd.to_numeric(df_resultados['Resultado'], errors='coerce')

# Todos os resultados

In [ ]:
df_resultados

#Melhores Resultados

In [ ]:
df_resultados.sort_values(by="Resultado", ascending=False).head(10)

In [ ]:
excel_file_path = "/content/resultados_mensais.xlsx"
df_resultados.to_excel(excel_file_path, index=False)